# Проведение A/B тестирования для New York City Taxi and Limousine Commission (TLC)

### Автор Андрей Самоленков

New York City TLC — это агентство, отвечающее за лицензирование и регулирование деятельности такси и сдаваемых в аренду транспортных средств в Нью-Йорке.

**Задача, которую поставило агенство New York City TLC:** проанализировать взаимосвязь между суммой тарифа и типом оплаты.

**Цель** — применить описательную статистику и проверку гипотез в Python. Целью этого A/B-теста является выборка данных и анализ наличия связи между типом платежа и суммой тарифа. Например: выяснить, платят ли клиенты, использующие кредитные карты, более высокие суммы за проезд, чем клиенты, использующие наличные.

**Примечание.** Для целей этого проекта выборочные данные пбыли олучены в результате эксперимента, в котором клиенты выбираются случайным образом и делятся на две группы: 1) клиенты, которым необходимо платить кредитной картой, 2) клиенты, которые необходимо оплатить наличными. 
  
*Этот проект состоит из следующих этапов:*

**Часть 1.** Импорт и загрузка данных.

**Часть 2:** Проведение EDA 

**Часть 3:** Проверка гипотез.

**Часть 4.** Обмен информацией с заинтересованными сторонами

### 1. Импорт библиотек и загрузка данных

In [5]:
import pandas as pd
import numpy as np
from scipy import stats

In [6]:
# Загрузка датасета в датафрейм
taxi_data = pd.read_csv("2017_Yellow_Taxi_Trip_Data.csv", index_col = 0)

### 2. Исследование данных

В столбце `payment_type` закодированы следующие значения:
*   1: Credit card (Кредитная карта)
*   2: Cash (Наличные)
*   3: No charge (Без оплаты)
*   4: Dispute (Спор)
*   5: Unknown (Неизвестно)



In [7]:
taxi_data.describe(include='all')

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
count,22699.000000,22699,22699,22699.000000,22699.000000,22699.000000,22699,22699.000000,22699.000000,22699.000000,22699.000000,22699.000000,22699.000000,22699.000000,22699.000000,22699.000000,22699.000000
unique,NaN,22687,22688,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,07/03/2017 3:45:19 PM,10/18/2017 8:07:45 PM,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,2,2,NaN,NaN,NaN,22600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,1.556236,NaN,NaN,1.642319,2.913313,1.043394,NaN,162.412353,161.527997,1.336887,13.026629,0.333275,0.497445,1.835781,0.312542,0.299551,16.310502
std,0.496838,NaN,NaN,1.285231,3.653171,0.708391,NaN,66.633373,70.139691,0.496211,13.243791,0.463097,0.039465,2.800626,1.399212,0.015673,16.097295
min,1.000000,NaN,NaN,0.000000,0.000000,1.000000,NaN,1.000000,1.000000,1.000000,-120.000000,-1.000000,-0.500000,0.000000,0.000000,-0.300000,-120.300000
25%,1.000000,NaN,NaN,1.000000,0.990000,1.000000,NaN,114.000000,112.000000,1.000000,6.500000,0.000000,0.500000,0.000000,0.000000,0.300000,8.750000
50%,2.000000,NaN,NaN,1.000000,1.610000,1.000000,NaN,162.000000,162.000000,1.000000,9.500000,0.000000,0.500000,1.350000,0.000000,0.300000,11.800000
75%,2.000000,NaN,NaN,2.000000,3.060000,1.000000,NaN,233.000000,233.000000,2.000000,14.500000,0.500000,0.500000,2.450000,0.000000,0.300000,17.800000


Меня интересует взаимосвязь между типом оплаты и суммой тарифа, которую платит клиент. Один из подходов — посмотреть среднюю сумму тарифа для каждого типа оплаты.

In [8]:
taxi_data.groupby('payment_type')['fare_amount'].mean()

payment_type
1    13.429748
2    12.213546
3    12.186116
4     9.913043
Name: fare_amount, dtype: float64

Судя по приведенным средним показателям, оказывается, что клиенты, расплачивающиеся кредитной картой, как правило, платят большую сумму за проезд, чем клиенты, которые платят наличными. Однако эта разница может возникнуть в результате случайной выборки, а не являться реальной разницей в сумме тарифа. Чтобы оценить, является ли разница статистически значимой, проведу проверку гипотезы.

### 3. Проверка гипотезы

Цель на этом этапе — провести t-тест для двух выборок. Этапы проведения проверки гипотезы выглядят следующим образом:


1. Формулируем нулевую гипотезу и альтернативную гипотезу.
2. Выбираем уровень значимости
3. Находим p-value
4. Принимаем решение: отвергнуть или не отвергнуть нулевую гипотезу.



### Нулевая гипотеза: Нет никакой разницы в средней сумме тарифа между клиентами, использующими кредитные карты, и клиентами, использующими наличные.

### Альтернативная гипотеза: Существует разница в средней сумме тарифа между клиентами, использующими кредитные карты, и клиентами, использующими наличные.

В качестве уровня значимости выбираем значение 5%

In [9]:
#hypothesis test, A/B test
#significance level
credit_card = taxi_data[taxi_data['payment_type'] == 1]['fare_amount']
cash = taxi_data[taxi_data['payment_type'] == 2]['fare_amount']
stats.ttest_ind(a=credit_card, b=cash, equal_var=False)

TtestResult(statistic=6.866800855655372, pvalue=6.797387473030518e-12, df=16675.48547403633)

Поскольку p-value значительно меньше уровня значимости 5%, я отвергаю нулевую гипотезу.

Прихожу к выводу, что существует статистически значимая разница в средней сумме тарифа между клиентами, использующими кредитные карты, и клиентами, использующими наличные.

### 4. Обмен информацией с заинтересованными сторонами

Ключевая бизнес-идея заключается в том, что поощрение клиентов к оплате кредитными картами может принести больше дохода водителям такси.

Этот проект предполагает то, что пассажиры были вынуждены платить тем или иным способом и что, узнав об этом требовании, они всегда его соблюдали. Данные не собирались таким образом; поэтому необходимо было сделать предположение о случайной группировке записей данных для проведения A/B-теста. Этот набор данных не учитывает другие вероятные объяснения. Например, пассажиры могут не брать с собой много наличных, поэтому за более длительные поездки проще платить кредитной картой. Другими словами, гораздо более вероятно, что сумма тарифа определяет тип оплаты, а не наоборот.

**Спасибо за внимание**